# 다운로드

In [9]:
# 모든 .tar.gz 압축 해제
!for f in *.tar.gz; do tar -xvf "$f"; done

noNewline_train_dataset/
noNewline_train_dataset/validation/
noNewline_train_dataset/validation/dataset_info.json
noNewline_train_dataset/validation/state.json
noNewline_train_dataset/validation/data-00000-of-00001.arrow
noNewline_train_dataset/train/
noNewline_train_dataset/train/dataset_info.json
noNewline_train_dataset/train/state.json
noNewline_train_dataset/train/data-00000-of-00001.arrow
noNewline_train_dataset/dataset_dict.json
wiki_noNewline_noFile.json

gzip: stdin: unexpected end of file
tar: Unexpected EOF in archive
tar: Unexpected EOF in archive
tar: Error is not recoverable: exiting now
wiki_noNewline_noFile_tokenized.bin
wiki_noNewline.json

gzip: stdin: unexpected end of file
tar: Unexpected EOF in archive
tar: Unexpected EOF in archive
tar: Error is not recoverable: exiting now
wiki_noNewline_tokenized.bin


# 세팅

In [5]:
import json
from datasets import load_from_disk, concatenate_datasets
from transformers import AutoTokenizer
from tqdm import tqdm
import re
import copy

/opt/conda/envs/mrc/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [23]:
# \n 제거한 데이터셋을 베이스로 사용
with open("data/wiki_noNewline.json", "r", encoding="utf-8") as f:
    wiki = json.load(f)
with open("data/wikipedia_documents.json", "r", encoding="utf-8") as f:
    pre_wiki = json.load(f)

In [7]:
train = load_from_disk("data/noNewline_train_dataset/train")
valid = load_from_disk("data/noNewline_train_dataset/validation")
test = load_from_disk("data/test_dataset/validation")
all = concatenate_datasets([train, valid])
len(train), len(valid), len(test), len(all)

(3952, 240, 600, 4192)

# utils

## evidence_ids: List[int]

In [8]:
# train/valid context의 document_id list

evidence_ids = []
evidence_ids.extend(set(all[:]["document_id"]))
len(evidence_ids), evidence_ids[:5]

(3504, [16385, 24578, 49158, 57351, 8208])

## check_evidence(List[str])

In [9]:
# check wiki key is in evidence ids

def check_evidence(keys: list):
    """
    return: in_keys (List[str]), out_keys (List[str])
    """
    if len(keys) > 0 and isinstance(keys[0], str):
        keys = [int(k) for k in keys]
    
    in_keys = []
    out_keys = []
    for k in keys:
        if k in evidence_ids: in_keys.append(k)
        else: out_keys.append(k)
    return {"in": in_keys, "out": out_keys}

check_evidence(["1000", "8208"])

{'in': [8208], 'out': [1000]}

## print_from_keys(List[str], wiki)

In [10]:
# print wiki text from keys
def print_from_keys(keys: list, context: dict):
    if len(keys) > 0 and isinstance(keys[0], int):
        keys = [str(k) for k in keys]
    for k in keys:
        print(k, "-"*20)
        print(context[k]["text"])
    print("-"*20)

# print_from_keys(["100", "1098"], wiki_noFile)

## find_key_to_change()

In [11]:
def find_key_to_change(cur_wiki, pattern, re_flags=re.MULTILINE, verbose=0):
    """
    verbose = 1(제거될 부분 출력), 2(기존 텍스트 출력)
    """
    changed_keys = []
    
    for key, item in cur_wiki.items():
        res = re.findall(pattern, item["text"], flags=re_flags)
        res = [r for r in res if re.search("[^\s]", r) ] # 공백으로만 이루어져 있는 span 제외
        if res:          
            changed_keys.append((key, res))
            
            if verbose >= 1:
                print(key, "-"*20)
                print(res)
            if verbose >= 2:
                print(item["text"])
    
    if verbose >= 1:        
        print("total:", len(changed_keys))
        print("in:", check_evidence(changed_keys)["in"])
    return changed_keys

## change_wiki()

In [12]:
def change_wiki(cur_wiki, pattern, key_to_change=[], re_flags=re.MULTILINE, verbose=0, del_threshold=50):
    """
    verbose=1(삭제될 텍스트 출력), 2(제거된 부분과 수정 후 텍스트 출력), 3(기존 텍스트 출력)
    """
    
    if len(key_to_change) == 0:
        print("-"*10, "No key_to_change provided. Got it from find_key_to_change()", "-"*10)
        key_to_change = find_key_to_change(cur_wiki, pattern, re_flags, verbose=0)
    
    changed_wiki = copy.deepcopy(cur_wiki)
    deleted_texts = []
    
    for (k, res) in key_to_change:
        text = cur_wiki[k]["text"]
        new_text = re.sub(pattern, "", text, flags=re_flags)
        changed_wiki[k]["text"] = new_text
        if verbose >= 2:
            print(k, "-"*20)
        if verbose >= 3:
            print(text)
        if verbose >= 2:
            print(res)
            print("-"*20)
            print(new_text)
        
        new_text = "".join(new_text.split())
        if len(new_text) < del_threshold:
            del changed_wiki[k]
            deleted_texts.append((k, new_text))
                
    if verbose >= 1:
        print("-"*10, f"Non-empty context that was deleted ({len(deleted_texts)})", "-"*10)
        for k, t in deleted_texts:
            if len(t) == 0: continue
            print(k,f"({len(t)})", "-"*20)
            print(t)
    
    print("-"*10, "Finished", "-"*10)
    print("in:", check_evidence(key_to_change)["in"])
    print(f"length: {len(cur_wiki)} -> {len(changed_wiki)}")
    return changed_wiki

## bm25plus

In [13]:
import os
import pickle
import time
from contextlib import contextmanager
from typing import List, Optional, Tuple, Union

import numpy as np
import pandas as pd
from datasets import Dataset #, concatenate_datasets, load_from_disk
from transformers import AutoTokenizer
# from sklearn.feature_extraction.text import TfidfVectorizer
from tqdm.auto import tqdm
# import wandb
# import datetime
# from pytz import timezone
from rank_bm25 import BM25Plus

@contextmanager
def timer(name):
    t0 = time.time()
    yield
    print(f"[{name}] done in {time.time() - t0:.3f} s")


class BM25PlusRetriever:
    def __init__(
        self,
        model_name_or_path: Optional[str] = "klue/bert-base",
        data_path: Optional[str] = "../data/",
        context_path: Optional[str] = "wikipedia_documents.json"
    ) -> None:
        """
        Arguments:

            data_path:
                데이터가 보관되어 있는 경로입니다.

            context_path:
                Passage들이 묶여있는 파일명입니다.

            data_path/context_path가 존재해야합니다.

        Summary:
            Passage 파일을 불러오고 BM25Plus를 선언하는 기능을 합니다.
        """

        self.data_path = data_path
        self.tokenizer = AutoTokenizer.from_pretrained(model_name_or_path) # default fast tokenizer if available
        self.contexts = None
        self.ids = None
        self.tokenized_contexts = None
        self.bm25plus = None
        
        context_name = context_path.split(".")[0]
        tokenized_wiki_path = os.path.join(data_path, context_name+"_tokenized.bin")
        print(f"context path:{os.path.join(data_path, context_path)}, tokenized context path: {tokenized_wiki_path}")        
        
        # contexts load
        with open(os.path.join(data_path, context_path), "r", encoding="utf-8") as f:
            wiki = json.load(f)

        # contexts 중복 제거
        self.contexts = list(
            dict.fromkeys([v["text"] for v in wiki.values()])
        )  # set 은 매번 순서가 바뀌므로
        print(f"Lengths of unique contexts : {len(self.contexts)}")
        self.ids = list(range(len(self.contexts)))
        
        # context 미리 토큰화
        # tokenized_wiki_path = {context_name}_tokenized.bin
        if os.path.isfile(tokenized_wiki_path):
            # 토큰화 된 context 파일 이미 존재
            with open(tokenized_wiki_path, "rb") as file:
                self.tokenized_contexts = pickle.load(file)
            print("Contexts pickle loaded.")
        else: 
            # 토큰화 된 context 파일 없으면 생성
            self.tokenized_contexts = []
            for doc in tqdm(self.contexts, desc="Tokenizing context"):
                self.tokenized_contexts.append(self.tokenizer(doc)["input_ids"][1:-1])
                # 앞 [CLS], 뒤  [SEP] 토큰 제외
            with open(tokenized_wiki_path, "wb") as f:
                pickle.dump(self.tokenized_contexts, f)
            print("Tokenized contexts pickle saved.")
            
        # Fit tokenized_contexts to BM25Plus
        self.bm25plus = BM25Plus(self.tokenized_contexts)
        print("Tokenized contexts fitted to BM25Plus.")
        

    def retrieve(
        self, query_or_dataset: Union[str, Dataset], topk: Optional[int] = 1, verbose: Optional[int] = 0
    ) -> Union[Tuple[List, List], pd.DataFrame]:
        """
        Arguments:
            query_or_dataset (Union[str, Dataset]):
                str이나 Dataset으로 이루어진 Query를 받습니다.
                각 query에 대해 `get_relevant_doc`을 통해 유사도를 구합니다.
            topk (Optional[int], optional): Defaults to 1.
                상위 몇 개의 passage를 사용할 것인지 지정합니다.

        Returns:
            1개의 Query를 받는 경우  -> Tuple(List, List) \n
            다수의 Query를 받는 경우 -> pd.DataFrame: [description]

        Note:
            다수의 Query를 받는 경우,
                Ground Truth가 있는 Query (train/valid) -> 기존 Ground Truth Passage를 같이 반환합니다.\n
                Ground Truth가 없는 Query (test) -> Retrieval한 Passage만 반환합니다.
        """

        if isinstance(query_or_dataset, str):
            doc_scores, doc_indices = self.get_relevant_doc(query_or_dataset, k=topk)
            if verbose >= 1:
                print("[Search query]\n", query_or_dataset, "\n")
                for i in range(topk):
                    print(f"Top-{i+1} passage with score {doc_scores[i]:4f}")
                    print(self.contexts[doc_indices[i]])

            return (doc_scores, [self.contexts[doc_indices[i]] for i in range(topk)])

        elif isinstance(query_or_dataset, Dataset):
            # Retrieve한 Passage를 pd.DataFrame으로 반환합니다.
            total = []
            with timer("BM25+ retrieval"):
                doc_scores = []
                doc_indices = []
                for query in tqdm(query_or_dataset["question"], desc="Retrieving documents for all questions"):
                    scores, indices = self.get_relevant_doc(query, k=topk)
                    doc_scores.append(scores)
                    doc_indices.append(indices)

            for idx, example in enumerate(
                tqdm(query_or_dataset, desc="Building result DataFrame")
            ):
                tmp = {
                    # Query와 해당 id를 반환합니다.
                    "question": example["question"],
                    "id": example["id"],
                    # Retrieve한 Passage의 id, context를 반환합니다.
                    "context": " ".join(
                        [self.contexts[pid] for pid in doc_indices[idx]]
                    ),
                }
                if "context" in example.keys() and "answers" in example.keys():
                    # validation 데이터를 사용하면 ground_truth context와 answer도 반환합니다.
                    tmp["original_context"] = example["context"]
                    tmp["answers"] = example["answers"]

                # 평가를 위한 df column 만들어주기
                for i in range(topk):
                    tmp["context" + str(i + 1)] = self.contexts[doc_indices[idx][i]]
                tmp['score'] = doc_scores[idx][:]

                total.append(tmp)

            cqas = pd.DataFrame(total)
            return cqas

    def get_relevant_doc(self, query: str, k: Optional[int] = 1) -> Tuple[List, List]:
        """
        Arguments:
            query (str):
                하나의 Query를 받습니다.
            k (Optional[int]): 1
                상위 몇 개의 Passage를 반환할지 정합니다.
        """

        tokenized_query = self.tokenizer(query)["input_ids"][1:-1]

        scores = [(val, idx) for idx, val in enumerate(self.bm25plus.get_scores(tokenized_query))]
        scores.sort(reverse=True)
        scores = scores[:k]
            
        doc_scores = [val for val, _ in scores]
        doc_indices = [idx for _, idx in scores]
            
        return doc_scores, doc_indices

In [75]:
# retriever = BM25PlusRetriever(
#     model_name_or_path="klue/bert-base",
#     data_path="data/",
#     context_path="wiki_noNewline.json"
# )

context path:data/wiki_noNewline.json, tokenized context path: data/wiki_noNewline_tokenized.bin
Lengths of unique contexts : 56737
Contexts pickle loaded.
Tokenized contexts fitted to BM25Plus.


# 전처리

## 파일: ... \n 삭제

In [38]:
# '파일: ... \n' 로만 이루어져 있는 줄 삭제
# 전처리한 wiki passage가 train/valid context로 쓰이지 않는다는 것 확인함

# 삭제한 후 남아있는 글자수(공백제외)가 50보다 작으면 해당 wiki 데이터 삭제
# 글자수, 삭제된 글
# 3, 예시:
# 19, <Gallery></gallery>
# 45, 제5항공모함항공단|link=VRC-30|VRC-30제5분견대*VRC-30제5분견대

pattern = "파일:[^\n]+\n|파일:[^\n]+\Z"

# for id in evidence_ids:
    # item = wiki[str(id)]
keys = []
for key, items in wiki.items():
    res = re.search(pattern, items["text"])
    if res:
        keys.append(key)
        # print(key, "-"*10, res)
        # print(items["text"])

print(len(keys), "/", len(wiki), "in:", check_evidence(keys)["in"])
# print_from_keys(keys[:2]) # 344, 1669

wiki_noFile = copy.deepcopy(wiki)

for k in keys:
    text = wiki[k]["text"]
    new_text = re.sub(pattern, "", text)
    wiki_noFile[k]["text"] = new_text
    # print(k, "-"*20)
    # print(text)
    # print("-->")
    # print(new_text)

    new_text = "".join(new_text.split())
    if len(new_text) < 50:
        del wiki_noFile[k]
        print(len(new_text), new_text)

print(len(wiki_noFile), len(wiki))

# with open("wiki_noNewline_noFile.json", "w") as f:
#     json.dump(wiki_noFile, f, ensure_ascii=False)
# !tar -czf wiki_noNewline_noFile.tar.gz wiki_noNewline_noFile.json

88 / 60613 in: []
0 
3 예시:
0 
0 
3 예시:
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
19 <Gallery></gallery>
0 
0 
0 
0 
0 
0 
0 
45 제5항공모함항공단|link=VRC-30|VRC-30제5분견대*VRC-30제5분견대
0 
60584 60613


## 한국어 아닌 문자로만 이루어져 있는 줄 삭제

In [265]:
cur_wiki = wiki_noFile
pattern = "^[^가-힣\n]+$"

key_to_change = find_key_to_change(cur_wiki, pattern)
wiki_korLine_noFile = change_wiki(cur_wiki, pattern, key_to_change=key_to_change, verbose=1, del_threshold=30)

---------- Non-empty context that was deleted (252) ----------
973 (28) --------------------
:참고문헌목록은제1차세계대전의서지문서를참조하십시오.
1163 (16) --------------------
쌍곡선함수의역함수는다음과같다.
1164 (9) --------------------
C는적분상수이다.
1667 (19) --------------------
니브흐어문자니브흐어는키릴문자를쓴다.
2815 (12) --------------------
:1절:2절:3절:4절
4382 (12) --------------------
::후렴::후렴::후렴
5527 (28) --------------------
:참고문헌목록은제1차세계대전의서지문서를참조하십시오.
5717 (16) --------------------
쌍곡선함수의역함수는다음과같다.
5718 (9) --------------------
C는적분상수이다.
6221 (19) --------------------
니브흐어문자니브흐어는키릴문자를쓴다.
7369 (12) --------------------
:1절:2절:3절:4절
8936 (12) --------------------
::후렴::후렴::후렴
9121 (9) --------------------
:1절:2절:3절
9193 (15) --------------------
//기본생성자(인자가없는).
9207 (18) --------------------
18판은총21개의장으로나뉘어있다.
9440 (11) --------------------
영어판참고문헌입니다.
9474 (11) --------------------
영어판참조문헌입니다.
9542 (14) --------------------
다음은영어판의참고문헌이다.
10674 (8) --------------------
스튜디오솔로음반
10743 (3) --------------------
결과:
10744 (27) ---------

In [266]:
# recover some
to_recover = [1667,10744, 15291]
for i in to_recover:
    wiki_korLine_noFile[str(i)] = copy.deepcopy(wiki[str(i)])
    wiki_korLine_noFile[str(i)]["text"] = re.sub(pattern, "", wiki[str(i)]["text"], flags=re.MULTILINE)
print(len(wiki_korLine_noFile))

60336


## url

In [286]:
pattern = "[|]*url[^ ]+$|url[.\n]*ref>"

wiki_noUrl = change_wiki(wiki_korLine_noFile, pattern, verbose=2)

---------- No key_to_change provided. Got it from find_key_to_change() ----------
11844 --------------------
['|url=http://www.numdam.org/item?id=MSM_1963__154__1_0|언어=fr}}</ref>']
--------------------
소인수 분해를 정수환에서 보다 일반적인 환으로 일반화하는 것은 환론의 오래된 문제이다. 일부 대수적 수체의 대수적 정수환이 유일 인수 분해 정역이 아니지만 (즉, 환의 원소가 기약원으로의 유일 인수 분해를 갖지 않을 수 있지만), 데데킨트 정역이라는 것(즉, 아이디얼이 소 아이디얼로의 유일한 분해를 갖는 것)이 밝혀지면서 환의 원소의 분해 대신 아이디얼의 분해가 대두되었다. 그러나 데데킨트 정역이 아닌 환들의 경우, 소 아이디얼로의 분해 역시 실패한다.

이를 해결하기 위하여, 에마누엘 라스커가 라스커-뇌터 정리를 다항식환에 대하여 증명하였고,  그 뒤 에미 뇌터가 라스커-뇌터 정리를 일반적 뇌터 가환환에 대하여 증명하였다. 44, §5, Satz IX 이에 따라 임의의 뇌터 가환환에 대하여 소인수 분해가 일반화되었다.

비가환환의 경우, 레옹스 르시외르(Léonce Lesieur)와 로베르 크루아조(Robert Croisot)가 삼종 아이디얼의 개념을 도입하여, 왼쪽 뇌터 환의 경우 삼종 분해가 성립함을 보였다.   anniversaire|저널=Journal für die reine und angewandte Mathematik|url=http://resolver.sub.uni-goettingen.de/purl?GDZPPN00217880X|권=204|날짜=1960|쪽=216–220|mr=0131436|doi=10.1515/crll.1960.204.216|issn=0075-4102|언어=fr}}</ref>|날짜=1963|총서=Mémorial des sciences mathématiques|권=154|mr=

In [298]:
pattern = "[(]http[^)]*[)]"
_ = find_key_to_change(wiki_noUrl, pattern, verbose=1)
wiki_noUrl = change_wiki(wiki_noUrl, pattern , verbose=1)


1458 --------------------
['(http://www.brianmayguitars.co.uk)']
3251 --------------------
['(https://www.bbc.com/sport/taekwondo/40391326#:~:text=The%20World%20Taekwondo%20Federation%20(WTF,it%20was%20established%20in%201973)']
6012 --------------------
['(http://www.brianmayguitars.co.uk)']
7805 --------------------
['(https://www.bbc.com/sport/taekwondo/40391326#:~:text=The%20World%20Taekwondo%20Federation%20(WTF,it%20was%20established%20in%201973)']
11494 --------------------
['(https://dowdistribution.co.kr)']
12313 --------------------
['(https://web.archive.org/web/20200418053617/https://www.fanfiction.net/crossovers/book)']
13072 --------------------
['(http:jehhs.co.kr/)']
17475 --------------------
['(https://web.archive.org/web/20181124042928/http://www.hansalim.or.kr/)', '(https://web.archive.org/web/20160617172938/http://icoop.coop/)', '(https://web.archive.org/web/20160611055202/http://www.ecoop.or.kr/)', '(https://web.archive.org/web/20160331135815/http://www.happycoop.o

In [312]:
pattern = "[\"]*http[^\s\|)\"]*[\"]*"
keys = find_key_to_change(wiki_noUrl, pattern, verbose=1)
wiki_noUrl = change_wiki(wiki_noUrl, pattern, key_to_change=keys, verbose=1)

246 --------------------
['https://ko.wikipedia.org/wiki/%ED%8C%8C%EC%9D%BC:Beethoven_3.jpg']
487 --------------------
['http://www.ethnologue.com/codes/#downloading']
1160 --------------------
['http://prkorea.com/writerttboard/tt/board/ttboard.cgi?act=read&db=centerop&idx=354#']
1261 --------------------
['http://article.joins.com/news/article/article.asp?total_id=3287561']
2786 --------------------
['http://www.kreml.ru/main.asp']
3162 --------------------
['http://www.nl.go.kr/nl/search/bookdetail/online.jsp?contents_id=CNTS-00069088866', 'http://www.nl.go.kr/nl/search/bookdetail/online.jsp?contents_id=CNTS-00069192278']
3821 --------------------
['https://m.blog.naver.com/colorcurator/221718015597', 'https://m.blog.naver.com/colorcurator/221718276320', 'https://m.blog.naver.com/colorcurator/221718029747', 'https://m.blog.naver.com/colorcurator/221717847497', 'https://m.blog.naver.com/colorcurator/221718264267', 'https://m.blog.naver.com/colorcurator/221718252141', 'https://m.blog.

In [313]:
to_recover = 37525
wiki_noUrl[str(to_recover)] = copy.deepcopy(wiki[str(to_recover)])
len(wiki_noUrl)

60334

## html tag

In [333]:
pattern = "<[^>]*/>"
keys = find_key_to_change(wiki_noUrl, pattern, verbose=1)
wiki_noTag = change_wiki(wiki_noUrl, pattern, key_to_change=keys, verbose=1)

446 --------------------
['<br />']
1254 --------------------
['<br/>']
1552 --------------------
['<br />', '<br />']
3162 --------------------
['<br/>', '<br/>', '<br/>', '<br/>']
5000 --------------------
['<br />']
5808 --------------------
['<br/>']
6106 --------------------
['<br />', '<br />']
7716 --------------------
['<br/>', '<br/>', '<br/>', '<br/>']
9304 --------------------
['<br/>']
10631 --------------------
['<br />', '<br />', '<br />']
14055 --------------------
['<br/>', '<br/>']
16845 --------------------
['<br/>']
17171 --------------------
['<br />']
21156 --------------------
['<br />', '<br />', '<br />', '<br />', '<br />', '<br />', '<br />', '<br />', '<br />', '<br />', '<br />', '<br />', '<br />', '<br />', '<br />', '<br />', '<br />', '<br />', '<br />', '<br />', '<br />', '<br />', '<br />', '<br />', '<br />', '<br />', '<br />', '<br />', '<br />', '<br />', '<br />', '<br />', '<br />', '<br />', '<br />', '<br />', '<br />', '<br />', '<br />', '<

In [334]:
pattern = "</[^>]*>"
keys = find_key_to_change(wiki_noTag, pattern, verbose=1)
wiki_noTag = change_wiki(wiki_noTag, pattern, key_to_change=keys, verbose=1)

357 --------------------
['</ref>']
358 --------------------
['</ref>']
517 --------------------
['</ref>']
717 --------------------
['</ref>']
726 --------------------
['</ref>']
739 --------------------
['</ref>']
742 --------------------
['</ref>']
754 --------------------
['</ref>']
758 --------------------
['</ref>']
781 --------------------
['</ref>']
1022 --------------------
['</ref>']
1160 --------------------
['</ref>']
1181 --------------------
['</ref>']
1261 --------------------
['</ref>']
1432 --------------------
['</br>']
1447 --------------------
['</br>']
1475 --------------------
['</ref>']
1809 --------------------
['</ref>', '</ref>']
1814 --------------------
['</ref>']
1815 --------------------
['</ref>']
1835 --------------------
['</ref>']
1836 --------------------
['</ref>']
2384 --------------------
['</ref>']
2396 --------------------
['</ref>']
2401 --------------------
['</ref>']
2407 --------------------
['</ref>']
2412 --------------------
['</ref>']
242

In [335]:
pattern = "</[^>]*>"
keys = find_key_to_change(wiki_noTag, pattern, verbose=1)
wiki_noTag = change_wiki(wiki_noTag, pattern, key_to_change=keys, verbose=1)

total: 0
in: []


## 공백 제외 글자수 적은 것 중 의미 없는 데이터 제거, meta data, 문맥 없는 단어 나열

In [326]:
for key, item in wiki_noTag.items():
    text = item["text"]
    text = " ".join(text.split())
    if len(text) < 50:
        print(key, f"({len(text)})", "-"*10)
        print(text)

875 (42) ----------
다음은 대화형으로 사용자의 숫자 입력을 받아서 평균을 출력하는 프로그램이다.
1687 (36) ----------
카탈랑 수의 n=0…37까지의 값들은 아래와 같다. A000108
3879 (41) ----------
원래 시는 12절이었으나, 1, 2, 5, 9, 12절만이 국가로 쓰인다.
4379 (42) ----------
! 독일어 가사 !! 프랑스어 가사 !! 이탈리아어 가사 !! 로만슈어 가사
5429 (42) ----------
다음은 대화형으로 사용자의 숫자 입력을 받아서 평균을 출력하는 프로그램이다.
6241 (36) ----------
카탈랑 수의 n=0…37까지의 값들은 아래와 같다. A000108
8433 (41) ----------
원래 시는 12절이었으나, 1, 2, 5, 9, 12절만이 국가로 쓰인다.
8933 (42) ----------
! 독일어 가사 !! 프랑스어 가사 !! 이탈리아어 가사 !! 로만슈어 가사
10874 (44) ----------
:후렴 :1절 :2절 :3절 :4절 :5절 :6절 :7절 :8절 :9절 :10절
13192 (38) ----------
|style="background:GOLD"|서울특별시 우승 (V1)
13527 (45) ----------
레코드셋 객체 기능을 사용하지 않고 SQL을 사용한 ASP 코드로는 다음과 같다.
17172 (47) ----------
L'últimu home 중 아스투리아스어 칸타브리아 번역본 카스티야어 번역본 영문판
24165 (49) ----------
Eternal Memory -소녀의 꿈- 고백, 꽃, 늑대 고백, 꽃, 늑대 Part 2
26385 (37) ----------
* United Kingdom Parliament. 공식 홈페이지.
27261 (40) ----------
XHTML 1.0도 HTML 4.01과 같이 세 가지 DTD로 구분된다.
27628 (41) ----------
아랍 문자는 1928년까지 소비에트 

In [336]:
a = [4379, 8933, 10874, 13192, 26385, 53129, 53131, 56353, 56572, 56806, 13507, 60574, 60293, 59382, 59344, 54169, 52090, 52088]
print(check_evidence(a)["in"])
print_from_keys(a, wiki)
for aa in a:
    try:
        del wiki_noTag[str(aa)]
    except: 
        print("failed", aa)
print(len(wiki_noTag))

[]
failed 53129
60316
